In [ ]:
!pip install astropy>=3.1
!pip install sunpy
!pip install imgaug==0.2.5
#!pip install stumpy
!pip install scipy --upgrade
#restart runtime after installation, it should work then
import pandas as pd
#import stumpy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import datetime as dt
import scipy
import sklearn
import copy

plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')

from google.colab import drive
drive.mount('/content/drive')
DATASET_PATH = '/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.0 MB 4.6 MB/s 
     |████████████████████████████████| 1.1 MB 10.2 MB/s 
     |████████████████████████████████| 144 kB 46.5 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 271 kB 52.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 562 kB 5.1 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-py3-none-any.whl size=561438 sha256=bc550e8516603a8facee928aa85dab1f96859b29aa780513497b5b510244cd15
  Stored in directory: /root/.cache/pip/wheels/60/dd/38/d1dc2cad2b6a66dc0249261004990bccb0f27985c74ba26e49
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
Looking in indexes: ht

In [ ]:
import time
start = 0*1440
n = start+1440
m=1

multiSeriesMain = pd.read_csv(DATASET_PATH+'4dadbc9a741ff86462d015981149d52bfcac6c1780e7ed6cd36bbd2664f398d0'+'.csv')
multiSeriesMain = multiSeriesMain.loc[:, ~multiSeriesMain.columns.str.contains('^Unnamed')].astype(float).reset_index(drop=True)
multiSeries = multiSeriesMain[start:n]

In [ ]:
#Euclidean Distance brute forcing
def GenDistancesBruteEuclidean(timeSeries):
  distances = []
  for s in range(len(timeSeries)-(m-1)):
    point1 = timeSeries.iloc[s:s+m, :].to_numpy().T
    temp = []
    for i in range(len(timeSeries)-(m-1)):
      point2 = timeSeries.iloc[i:i+m, :].to_numpy().T
      euclideanDist = np.linalg.norm(point1 - point2)
      temp.append(euclideanDist)
    distances.append(temp)
  return np.array(distances)

In [ ]:
distances = GenDistancesBruteEuclidean(multiSeries)

In [ ]:
distances = pd.DataFrame(distances)
print(distances.describe())

              0            1            2            3            4     \
count  1440.000000  1440.000000  1440.000000  1440.000000  1440.000000   
mean     13.069574    12.065426    11.334831    13.149781    13.695707   
std       7.328212     6.304015     7.969588     9.133474     5.772243   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       8.831761     7.937254     6.480741     6.782330    10.037407   
50%      11.401754    10.954451     9.165151     9.898979    13.038405   
75%      14.645789    14.142136    13.341664    17.088007    17.044042   
max      70.526591    68.658576    72.910905    75.921012    64.233947   

              5            6            7            8            9     ...  \
count  1440.000000  1440.000000  1440.000000  1440.000000  1440.000000  ...   
mean     12.419917    15.720344    11.773936    15.604991    13.938175  ...   
std       8.429397     6.117903     8.888201    11.285767     8.363248  ...   
min       0.00000

In [ ]:
#Neighbor Chain Grouping
def NeighborChainGrouping(distances, combinations=1):
  distances_copy = copy.deepcopy(distances)
  #remove self matches
  for row in range(len(distances_copy)):
    for col in range(len(distances_copy)):
      if row == col:
        distances_copy[row][col] = 9999999999999
  nearestNeighbors = []
  #initial nearest neighbor calculations
  for row in range(len(distances_copy)):
    nearestNeighbors.append([row, distances_copy.iloc[:, row].idxmin()])
  groupings = []
  for n in range(len(nearestNeighbors)):
    flag1 = False
    flag2 = False
    if len(groupings) == 0:
      if n == 0:
        groupings.append([nearestNeighbors[n][0], nearestNeighbors[n][1]])
    else:
      if any(n in sublist for sublist in groupings):
        flag1 = True
      if any(nearestNeighbors[n][1] in sublist for sublist in groupings):
        flag2 = True
      if flag1 and flag2:
        index1 = None
        index2 = None
        for g in range(len(groupings)):
          if n in groupings[g]:
            index1 = g
          if nearestNeighbors[n][1] in groupings[g]:
            index2 = g
        if index1 != index2:
          newList = groupings[index1] + groupings[index2]
          if index1 > index2:
            groupings.pop(index1)
            groupings.pop(index2)
          else: 
            groupings.pop(index2)
            groupings.pop(index1)
          groupings.append(newList)
      elif flag1:
        index = None
        for g in range(len(groupings)):
          if n in groupings[g]:
            index = g
        groupings[index].append(nearestNeighbors[n][1])
      elif flag2:
        index = None
        for g in range(len(groupings)):
          if nearestNeighbors[n][1] in groupings[g]:
            index = g
        groupings[index].append(nearestNeighbors[n][0])
      else:
        groupings.append([nearestNeighbors[n][0], nearestNeighbors[n][1]])
  return groupings
print(NeighborChainGrouping(distances))
print(distances.iloc[8, 0])

294
11.74734012447073
